# Análise de dados de temperatura coletados de uma API

Fonte dos dados: [https://www.ncdc.noaa.gov/cdo-web/token](https://www.ncdc.noaa.gov/cdo-web/token)

Bibliotecas importadas

In [2]:
import requests
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

plt.style.use('seaborn')

In [3]:
def make_request(endpoint, payload=None):
    """
    Faz uma requisição para a API de clima
    Parâmetros:
        - endpoint
        - payload
    Returns:
        Um objeto response
    """
    with open('../../TOKENS/ncdc_token.txt') as f:
        ncdc_tk = f.read()
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={'token': ncdc_tk},
        params=payload
    )

In [4]:
response = make_request('datasets', {'startdate': '2018-10-01'})

In [5]:
response.status_code

200

In [6]:
response.ok

True

Uma vez obtida a resposta, o método `json` pode ser utilizado para obter o dicionário dos dados a partir da variável `payload`:

In [7]:
payload = response.json()
payload.keys()

dict_keys(['metadata', 'results'])

In [8]:
payload['metadata']

{'resultset': {'offset': 1, 'count': 11, 'limit': 25}}

A chave `results` contém uma lista de dicionários. Por exemplo, o primeiro desses dicionários por sua vez, contém:

In [9]:
payload['results'][0].keys()

dict_keys(['uid', 'mindate', 'maxdate', 'name', 'datacoverage', 'id'])

Do resultado acima, ficamos com os campos referentes ap ID e ao nome dos _datasets_ (conjunto de dados):

In [10]:
[(data['id'], data['name']) for data in payload['results']]

[('GHCND', 'Daily Summaries'),
 ('GSOM', 'Global Summary of the Month'),
 ('GSOY', 'Global Summary of the Year'),
 ('NEXRAD2', 'Weather Radar (Level II)'),
 ('NEXRAD3', 'Weather Radar (Level III)'),
 ('NORMAL_ANN', 'Normals Annual/Seasonal'),
 ('NORMAL_DLY', 'Normals Daily'),
 ('NORMAL_HLY', 'Normals Hourly'),
 ('NORMAL_MLY', 'Normals Monthly'),
 ('PRECIP_15', 'Precipitation 15 Minute'),
 ('PRECIP_HLY', 'Precipitation Hourly')]

In [11]:
response = make_request(
    'datacategories', payload={'datasetid': 'GHCND'}
)

response.status_code

200